In [142]:
import pickle
import json
import sys
import os
import string
import requests
import ast  # for string to list: ast.literal_eval()
from bs4 import BeautifulSoup
import bs4
import time
from datetime import datetime
from datetime import date
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
import warnings
data_dir = "/Users/vigadam/Documents/github/media_network/data/origo/"

In [143]:
class Helper:
    import json

    from bs4 import BeautifulSoup
    import bs4

    import pandas as pd
    import numpy as np

    import re

    import warnings

    warnings.filterwarnings("ignore")

    def __init__(self, df):
        None

    def title_main(self, element, id_):
        for i, soup in tqdm(enumerate(self.soups)):
            try:
                df.loc[self.soups.index[i], "Title"] = (
                    BeautifulSoup(soup).find(attrs={element: id_}).get_text()
                )
            except AttributeError:
                df.loc[self.soups.index[i], "Title"] = None

    def link(self):
        for i, soup in tqdm(enumerate(self.soups)):
            soup_for_links = BeautifulSoup(soup).find_all("a")
            link_list = []
            for item in soup_for_links:
                if type(item.get("href")) == str:
                    link_list.append(item.get("href"))
            df.loc[self.soups.index[i], "links"] = str(list(set(link_list)))

    def text_main(self, element, class_id):
        for i, soup in tqdm(enumerate(self.soups)):
            try:
                soup_for_text = (
                    BeautifulSoup(soup)
                    .find(element, class_=re.compile(class_id))
                    .find_all("p")
                )
                text_list = []
                for item in soup_for_text:
                    if soup_for_text != "":
                        text_list.append(item.text)
                df.loc[self.soups.index[i], "Text"] = " ".join(text_list)
            except AttributeError:
                df.loc[self.soups.index[i], "Text"] = None

    def author_main(self, element, class_id):
        for i, soup in tqdm(enumerate(self.soups)):
            try:
                df.loc[self.soups.index[i], "Author"] = (
                    BeautifulSoup(soup)
                    .find(attrs={element:class_id})
                    .get_text()
                    .strip()
                )
            except:
                df.loc[self.soups.index[i], "Author"] = None

    def category_main(self, element, id_):
        for i, soup in tqdm(enumerate(self.soups)):
            try:
                df.loc[self.soups.index[i], "Category"] = (
                    BeautifulSoup(soup).find(attrs={element: id_}).text.strip()
                )

            except:
                df.loc[self.soups.index[i], "Category"] = None

    def tags_main(self, element, id_):
        for i, soup in tqdm(enumerate(self.soups)):
            try:
                df.loc[self.soups.index[i], "Tags"] = (
                    BeautifulSoup(soup)
                    .find(attrs={element: id_}).text.strip()
                )

            except:
                df.loc[self.soups.index[i], "Tags"] = None
        df["Tags"] = df["Tags"].str.split("\n\n\n")

    def clean(self):
        self.title()
        self.text()
        self.link()
        self.author()
        self.tags()

In [144]:
class Origo(Helper):
    def __init__(self, df):
        page = "origo"
        self.data = df.loc[df["page"] == page]
        self.soups = self.data["soup"].dropna()

    def title(self):
        Helper.title_main(self, "class", "article-title")

    def text(self):
        for i, soup in tqdm(enumerate(self.soups)):
            try:
                textlist = BeautifulSoup(soup).find(
                    attrs={"class": "article-lead"}
                ).find_all("p") + BeautifulSoup(soup).find(
                    attrs={"class": "article-content"}
                ).find_all(
                    "p"
                )

                df.loc[self.soups.index[i], "Text"] = (
                    " ".join([item.get_text().strip() for item in textlist])
                    .replace("\xa0", "")
                    .strip()
                )
            except AttributeError:
                df.loc[self.soups.index[i], "Text"] = None

    def author(self):
        Helper.author_main(self, "class", "article-author")

    def tags(self):
        for i, soup in tqdm(enumerate(self.soups)):
            try:
                df.loc[self.soups.index[i], "Tags"] = str([
                    item.get("title")
                    for item in BeautifulSoup(soup)
                    .find_all(attrs={"class": "article-meta"})[0]
                    .find_all("a")[1:]
                ])

            except:
                df.loc[self.soups.index[i], "Tags"] = None

In [101]:
df = pd.read_pickle(data_dir + "month_01_soups.pkl").drop(columns=["Unnamed: 0"])
df["page"] = "origo"

In [122]:
soup = df.loc[15, "soup"]

In [123]:
[
    item.get("title")
    for item in BeautifulSoup(soup)
    .find_all(attrs={"class": "article-meta"})[0]
    .find_all("a")[1:]
]

['jég', 'Bajkál-tó', 'autós olvasnivaló', 'Lexus', 'Oroszország']

In [121]:
with open(data_dir + "file.html", "w") as file:
    file.write(df.loc[15, "soup"])

In [102]:
origo = Origo(df)
#origo = Origo(df.loc[0:40,:])

In [83]:
data_list = sorted([
    "month_04_soups.pkl",
    "month_12_soups.pkl",
    "month_03_soups.pkl",
    "month_06_soups.pkl",
    "month_07_soups.pkl",
    "month_02_soups.pkl",
    "month_10_soups.pkl",
    "month_08_soups.pkl",
    "month_05_soups.pkl",
    "month_11_soups.pkl",
    "month_01_soups.pkl",
    "month_09_soups.pkl",
])

data_files = [data_dir + item for item in data_list]

In [141]:
df_clean = pd.DataFrame()
for data_month in tqdm(data_files):

    df = pd.read_pickle(data_month)
    df["page"] = "origo"

    Origo(df).clean()

    df_clean = pd.concat([df_clean, df.drop(columns=["Unnamed: 0", "soup"])])

Exception ignored in: <generator object tqdm.__iter__ at 0x113fc77b0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/tqdm/std.py", line 1186, in __iter__
    self.close()
  File "/usr/local/lib/python3.8/site-packages/tqdm/notebook.py", line 251, in close
    super(tqdm_notebook, self).close(*args, **kwargs)
  File "/usr/local/lib/python3.8/site-packages/tqdm/std.py", line 1280, in close
    self._decr_instances(self)
  File "/usr/local/lib/python3.8/site-packages/tqdm/std.py", line 574, in _decr_instances
    cls.monitor.exit()
  File "/usr/local/lib/python3.8/site-packages/tqdm/_monitor.py", line 53, in exit
    self.join()
  File "/usr/local/Cellar/python@3.8/3.8.6/Frameworks/Python.framework/Versions/3.8/lib/python3.8/threading.py", line 1011, in join
    self._wait_for_tstate_lock()
  File "/usr/local/Cellar/python@3.8/3.8.6/Frameworks/Python.framework/Versions/3.8/lib/python3.8/threading.py", line 1027, in _wait_for_tstate_lock
    elif lock.acq

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-141-cdccbc8b8780>", line 7, in <module>
    Origo(df).clean()
  File "<ipython-input-91-3d241ad20825>", line 88, in clean
    self.title()
  File "<ipython-input-130-4a169155f68c>", line 8, in title
    Helper.title_main(self, "class", "article-title")
  File "<ipython-input-91-3d241ad20825>", line 23, in title_main
    BeautifulSoup(soup).find(attrs={element: id_}).get_text()
  File "/usr/local/lib/python3.8/site-packages/bs4/__init__.py", line 345, in __init__
    self._feed()
  File "/usr/local/lib/python3.8/site-packages/bs4/__init__.py", line 431, in _feed
    self.builder.feed(self.markup)
  File "/usr/local/lib/python3.8/site-packages/bs4/builder/_lxml.py", line 324, in feed
    self.parser.feed(markup)
  File "src/lxml/parser.pxi", line 1256, in lxml.etree._FeedPar

TypeError: object of type 'NoneType' has no len()

In [109]:
df_clean.to_pickle(data_dir + "origo_clean.pkl")

In [154]:
df = pd.read_pickle(data_dir + "origo_clean.pkl").reset_index(drop=True)

In [149]:
df.head()

,link,date,month,rovat,page,Title,Text,links,Author,Tags
0,https://www.origo.hu/tudomany/20200107-cica-mo...,20200107,01,tudomany,origo,A dorombolás gyógyító ereje,"Az általános hipotézis az, hogy a macska elége...",['https://www.origo.hu/cimkek/podcast/index.ht...,Szomor Anikó,"['etológia', 'állatvédelem', 'állatorvoslás', ..."
1,https://www.origo.hu/sport/egyeni/20200129-buc...,20200129,01,sport,origo,None,None,"['/sport/loero/index.html', 'https://reblog.hu...",Zámbó István,None
2,https://www.origo.hu/itthon/20200127-az-i-keru...,20200127,01,itthon,origo,Egymás után próbálja megakadályozni a fejleszt...,Az I. kerületben az októberi választások után ...,['https://www.origo.hu/cimkek/varkapitanysag/i...,Szalma György,"['Budai Vár', 'Budavár', 'V. Naszályi Márta', ..."
3,https://www.origo.hu/sport/egyeni/20200120-par...,20200120,01,sport,origo,None,None,['https://www.instagram.com/p/B7PEJolp4Qh/?utm...,Hegyi Áron,None
4,https://www.origo.hu/itthon/20200127-a-zoldkon...,20200127,01,itthon,origo,A környezetvédő konzervatívokat támadják a tém...,A konzervatív jobboldali politikusok által kín...,['https://www.origo.hu/cimkek/podcast/index.ht...,Kurmai Kornél,"['környezetvédelem', 'klímavédelem', 'ökofasis..."


In [150]:
df["rovat"].value_counts().head(20)

itthon        16822
sport         14582
nagyvilag      9142
gazdasag       6040
techbazis      3361
tudomany       3352
auto           1622
filmklub       1540
tafelspicc     1142
idojaras          3
Name: rovat, dtype: int64

In [158]:
df.loc[1310,"Text"]

'Összesen 17,5 millió dollárnyi büntetést kell fizetnie John Stumpfnak, a Wells Fargo bank korábbi vezérigazgatójának, mivel kiderült, a vállalat több millió hamis számlát kreált az ügyfelek felhatalmazása nélkül. Ritkán fordul elő a pénzügyi szektorban, hogy egy bankvezetőt személyesen büntetnek meg a hatóságok azért, mert az általa irányított pénzintézet törvénytelenül járt el. Most azonban az amerikai Wells Fargo befektetési bank volt vezetőjének, John Stumpfnak 17,5 millió dollárt kell fizetnie, amiért bankja hamis számlákat gyártott– írja a BBC. A vádakat azt követően fogalmazták meg, hogy kiderült, a Wells Fargonál több millió hamis ügyfélszámlát hoztak létre annak érdekében, hogy tartani tudják az értékesítési célokat. A vállalat közlése szerint mintegy 3,5 millió ilyen számlát kreáltak, az ügyfelek engedélye nélkül. A mostani döntés értelmében John Stumpf soha többé, semmilyen formában nem dolgozhat a pénzügyi szektorban. A szövetségi bankrendszert adminisztráló amerikai hatósá

In [152]:
14600/57600

0.2534722222222222